# 1. Exploratory data analysis - data cleaning and transformation
- Data cleaning is performed to convert columns to the correct format, handle nulls, and deal with missing values and outliers in the data

## 1.1 Imports and data loading
- Required modules are imported along with required classes defined in other files
- Data is loaded from .csv file into a pd dataframe 
- An instance of the DataTransform class is initialised for data transformation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px 
import seaborn as sns
from datetime import datetime 
from db_datatransform import DataTransform
from db_info import DataFrameInfo
from db_info import read_csv
from db_clean_data import Plotter
from db_clean_data import DataFrameTransform
from db_datatransform import DataTransform
from db_info import DataFrameInfo
from scipy import stats

In [ ]:
filename = 'loan_payments.csv'
loan_payments_df = read_csv(filename)  # calls the read_csv funciton to load data 
loan_payments_df = loan_payments_df.drop(columns=loan_payments_df.columns[0], axis=1) # remove additional index column
column_names = loan_payments_df.columns.tolist() # creates a list of the column headings as strings 
data_transform_instance = DataTransform(loan_payments_df) # creates an instance of the DataTransform class

## 1.2 Converting columns to the correct format
- Columns identified as being the wrong data type are transformed using methods created in the DataTransform class\
- Columns are listed and then converted to datetime, float, string or categorical

In [ ]:
col_to_convert_to_datetime = ['issue_date', 
                                 'earliest_credit_line', 
                                 'last_payment_date',
                                 'next_payment_date', 
                                 'last_credit_pull_date'] # list of strings specifying columns to be converted
col_to_convert_to_float = ['term','employment_length'] # list of strings of column names
col_to_convert_to_str = ['grade', 'sub_grade'] # list of strings of column names
col_to_convert_to_categorical = ['home_ownership',
                                    'verification_status', 
                                    'loan_status', 
                                    'payment_plan', 
                                    'purpose', 
                                    'application_type']  # list of strings of column names

In [ ]:
for i in range(0,len(col_to_convert_to_datetime)): # loops over list of column names
       datetime_format = '%b-%Y'
       data_transform_instance.obj_to_datetime(col_to_convert_to_datetime[i],datetime_format) # calls the obj_to_datetime method to convert data to datetime

In [ ]:
for i in range(0,len(col_to_convert_to_float)): # loops over list of column names
       data_transform_instance.obj_to_int(col_to_convert_to_float[i]) # calls the obj_to_int method to convert data to int

In [ ]:
for i in range(0,len(col_to_convert_to_str)): # loops over list of column names
       data_transform_instance.obj_to_str(col_to_convert_to_str[i]) # calls the obj_to_string method to convert data to str

In [ ]:
for i in range(0,len(col_to_convert_to_categorical)): #loops over list of column names
       data_transform_instance.obj_to_cat(col_to_convert_to_categorical[i]) # calls the obj_to_ method to convert data to categorical

## 1.3 Handling Nulls
- Null values are identified using methods from the DataFrameInfo class
- Null values are handled either by dropping the entire column, imputing the data, or dropping null values depending on the column type and percentage of null values 


### 1.3.1 Identifying null values 
- Null values are identified using the null_percentage method of the DataFrameInfo class, columns with null values are appended to a list

In [ ]:
nulls_info_instance = DataFrameInfo(loan_payments_df) # initialises an instnce of the class to obtain information on the dataframe
null_columns = [] # initialise list of null columns
for i in range (0, len(column_names)):
    null_pc = nulls_info_instance.null_percentage(column_names[i])
    if null_pc > 0.0:
        null_columns.append(column_names[i]) # append to list of columns if there are any null values 

In [ ]:
for i in range (0, len(column_names)):
    null_pc = nulls_info_instance.null_percentage(column_names[i])
    print(column_names[i], null_pc)
print(null_columns)

### 1.3.2 Handling null values
- Columns with a high percentage of null values (>50%) are dropped
- Columns with a small amount (1<%<49) of null values will be handled by imputing values, as dropping these rows could cause information loss
- Columns with a very small amount (<1%) of null values will have rows containing null values dropped

- Columns and rows containing null values are dropped using the drop_column and drop_null_rows methods of the DataFrameTransform class
- Descriptive stastics are calculated for columns that will be imputed to determine if missing values are imputed by the mean, mode or median value
- Values are imputed using methods from the DataFrameTransform class

In [ ]:
columns_to_drop = ['mths_since_last_delinq', 
                      'mths_since_last_record', 
                      'next_payment_date', 
                      'mths_since_last_major_derog'] # list of strings of columns with > 50% null values
columns_to_impute = ['funded_amount','term','int_rate','employment_length'] # list of strings of columns with a small amount of null values 
columns_to_drop_null_value_rows = ['last_payment_date', 
                                      'last_credit_pull_date', 
                                      'collections_12_mths_ex_med'] # list of strings of columns with < 1% null values 

In [ ]:
loan_payments_df_copy = loan_payments_df.copy() #create copy of df before removing any values for data preservation
remove_null_instance = DataFrameTransform(loan_payments_df_copy) # initialises an instance of the DataFrameTransform class for dealing with null values
df_info_instance = DataFrameInfo(loan_payments_df_copy) # initialises an instnce of the class to get info on the df 

In [ ]:
for i in range(0, len(columns_to_drop)):
      loan_payments_df_copy = remove_null_instance.drop_column(columns_to_drop[i]) # drops columns with > 50 % null values
for i in range(0, len(columns_to_drop_null_value_rows)):
      loan_payments_df_copy = remove_null_instance.drop_null_rows(columns_to_drop_null_value_rows[i]) # drops rows with null values in specific columns

In [ ]:
for i in range(0, len(columns_to_impute)):
      mean = df_info_instance.get_mean(columns_to_impute[i])
      print('MEAN', columns_to_impute[i], mean)

In [ ]:
for i in range(0, len(columns_to_impute)):
      stdv = df_info_instance.get_stdev(columns_to_impute[i])
      print('STDEV', columns_to_impute[i], stdv)

In [ ]:
for i in range(0, len(columns_to_impute)):
       mode = df_info_instance.get_mode(columns_to_impute[i])
       print('MODE', columns_to_impute[i], mode)

In [ ]:
for i in range(0, len(columns_to_impute)):
       median = df_info_instance.get_median(columns_to_impute[i])
       print('MEDIAN', columns_to_impute[i], median)

In [ ]:
for i in range(0, len(columns_to_impute)):
       ranges = df_info_instance.get_range(columns_to_impute[i])
       print('RANGE',columns_to_impute[i], ranges)

In [ ]:
for i in range(0, len(columns_to_impute)):
       df_info_instance.get_normal_dist(columns_to_impute[i])

In [ ]:
loan_payments_df_copy = remove_null_instance.impute_na_with_mode('term') # impute null values with mode, as this is categorical data
loan_payments_df_copy = remove_null_instance.impute_na_with_median('employment_length') # impute null value with median, to keep all values as whole numbers 
loan_payments_df_copy = remove_null_instance.impute_na_with_mean('funded_amount') # impute null values with mean as data is continuous with a normal distribution 
loan_payments_df_copy = remove_null_instance.impute_na_with_mean('int_rate') # impute null values with mean as data is continuous with a normal distribution

### 1.3.3 Checking null values again
- Null values are checked again after the identified nulls have been dropped or imputed to ensure no null values remain

In [ ]:
column_names_copy = loan_payments_df_copy.columns.tolist() # creates a list of the remaining column headings as strings 
null_columns_copy = []
for i in range (0, len(column_names_copy)): # to check that the correct columns and rows have been dropped and view null percentages to confirm 
    null_pc = df_info_instance.null_percentage(column_names_copy[i])
    print(column_names_copy[i], null_pc)
    if null_pc > 0.0:
      null_columns_copy.append(column_names_copy[i]) # append columns with null values to list 
print(null_columns_copy) # check print to ensure correct columns and rows have been dropped 

## 1.4 Identifying skew
- Skewed columns are identified through data visualisation using methods from the Plotter class
- The skew of each column is also assigned a numerical value calculated using statistical methods
- Columns with a numeric skew value <-2 or >2 are defined as skewed columns


### 1.4.1 Data visualisation
- Data is visualised for each column using historgrams and KDE plots generated from the Plotter class

In [ ]:
plotter_instance = Plotter(loan_payments_df_copy) # initialise an instance of the plotter class for data visualisation

In [ ]:
 # visualise data to observe skew
for i in range(0, len(column_names_copy)):
      plotter_instance.plot_hist(column_names_copy[i])

#for i in range(0, len(column_names_copy)):
 #     plotter_instance.plot_KDE(column_names_copy[i])

In [ ]:
loan_df_skew = loan_payments_df_copy.skew(axis=0,numeric_only = True) # obtain the skew of each numeric column in the dataframe
print(loan_df_skew)
check_skewed_columns = loan_df_skew.index
skewed_columns = []
for i in range(0, len(loan_df_skew)): 
      if loan_df_skew.iloc[i] > 2 or loan_df_skew.iloc[i] < -2: # append columns with a skew < -2 or > 2 to the list
         skewed_columns.append(check_skewed_columns[i])
print(skewed_columns) # show skewed columns

### 1.4.2 Skewed columns to ignore
- Skewed columns that are not appropriate candidates for transformation are ignored and removed from skewed_columns
- Skewed columns with a majority of 0 values are ignored
- Skewed columns containing categroical data or ID data are ignored

In [ ]:
skewed_columns_to_ignore = ['id','member_id','delinq_2yrs','inq_last_6mths','collections_12_mths_ex_med'] # skewed columns that do not need to be transformed
zero_maj_skewed_columns_to_ignore = ['out_prncp','out_prncp_inv','total_rec_late_fee','collection_recovery_fee'] # list of columns containing a majority of 0 values
for i in range(0, len(skewed_columns_to_ignore)):
      skewed_columns.remove(skewed_columns_to_ignore[i]) # removes columns that represent IDs/categorical data from list of skewed columns
for i in range(0, len(zero_maj_skewed_columns_to_ignore)):
      skewed_columns.remove(zero_maj_skewed_columns_to_ignore[i]) # removes columns that contain a majority of 0 values, meaning transformations are not appropriate
print(skewed_columns) # check to ensure correct cols are removed

### 1.4.3 Skew transformations 
- Skewed columns are identified, but no skew transformations are performed to enable querying and analysis of the data in the next section.

## 1.5 Identifying and handling outliers
- Outliers are identified using data visualisation methods from the Plotter class 
- Outliers are handled using methods from the DataFrameTransform class

### 1.5.1 Data visualisation
- Data is visualised for each column using historgrams and KDE plots generated from the Plotter class

In [ ]:
loan_payments_df_transformed = loan_payments_df_copy.copy() # create copy of transformed data with appropriate naming
transform_instance = DataFrameTransform(loan_payments_df_transformed) # initialise an instance of the class with the transformed dataframe
transformed_plotter_instance = Plotter(loan_payments_df_transformed) # initialise an instance of the plotter class for data visualisation

In [ ]:
 # visualise data to identify outliers 
for i in range(0, len(column_names_copy)):
      transformed_plotter_instance.plot_hist(column_names_copy[i])

#for i in range(0, len(column_names_copy)):
 #     transformed_plotter_instance.plot_KDE(column_names_copy[i])

### 1.5.2 Handling outliers
- Several outliers that were far higher than the normal distribution of data were identified, these were removed using the remove_top_val method of the DataFrameTransform class
- Negative outliers were identified in columns where context specific knowledge indicated that negative values were not logical, these were removed using the remove_negatives method of the DataFrameTransform class

In [ ]:
# remove outliers observed in visualisation of data 
columns_with_max_outliers = ['total_rec_late_fee','open_accounts','total_accounts','collection_recovery_fee'] # list of strings of column names for columns with high values to remove
for i in range(0, len(columns_with_max_outliers)):
      loan_payments_df_transformed = transform_instance.remove_top_val(columns_with_max_outliers[i]) # remove max values
# additional to remove highest val again
loan_payments_df_transformed = transform_instance.remove_top_val('collection_recovery_fee') 
loan_payments_df_transformed = transform_instance.remove_top_val('total_rec_late_fee') 

columns_with_negatives = ['recoveries','last_payment_amount'] # list of strings of column names for columns with -ve values to remove
for i in range(0, len(columns_with_negatives)):
      loan_payments_df_transformed = transform_instance.remove_negatives(columns_with_negatives[i]) # remove negative values

## 1.6 Correlated columns
- Overly correlated columns are identified with a correlation heatmap but no correlated columns are dropped in order to enable querying and analysis of the full dataset

In [ ]:
transformed_plotter_instance.plot_corr_matrix() # visualise correlated columns

## 1.7 Saving the cleaned data
- Data cleaning has been completed and the cleaned data is saved to a pickle file for data preservation 

In [ ]:
loan_payments_df_transformed.to_pickle('cleaned_data.pickle') # saves data in pickle format to preserve data types/data transformations 

# 2. Data analysis and visualisation
- Further analysis is performed on the data to gain deeper insights now the data has been transformed
- Data is loaded to a dataframe from the saved pickle file 

In [ ]:
loan_payments_df = pd.read_pickle('cleaned_data.pickle') # load in cleaned and transformed data from pickle file
cleaned_data_plotter_instance = Plotter(loan_payments_df)   # initialise an instance of the plotter class with cleaned data
column_names = loan_payments_df.columns.tolist() # create list of column names for analysis

### 2.1.1 Summarise what % of loans are recovered against investor funding and total amount funded
- Recovered loans are defined as those where the outstanding principal is 0 
- Percentage is calculated as a percentage of total loans issued
- EDA revealed a 1:1 correlation between out_prncp and out_prncp_inv, percentage recovered is therefore the same for both measures
- Results are visualised as a bar plot 

In [ ]:
# Summarise currently what percentage of the loans are recovered against the investor funding and the total amount funded
out_prncp_zeros = (loan_payments_df.out_prncp == 0.00).sum()  
loans_recov_against_total_funding = (out_prncp_zeros/(len(loan_payments_df)))*100
print(f"Percentage of the loans recovered against the investor funding and the total amount funded = {loans_recov_against_total_funding.round(2)} %")
loans_recov_against_inv_funding = loans_recov_against_total_funding # out_prncp_inv is same % as columns have 1:1 correlation
funding = ['total_funding','investor_funding']
recovery_pc = [loans_recov_against_total_funding, loans_recov_against_inv_funding]
funding_df = pd.DataFrame({'Funding': funding, 'Recovery_percentage': recovery_pc}) # load data into df to enable plotting
fig1 = plt.figure(1)
plt.bar(funding_df['Funding'], funding_df['Recovery_percentage'], width=0.8) # plot bar chart of % recovery
plt.xticks(rotation=90)

### 2.1.2 Visualise what % of the total amount would be recovered up to 6 months in the future
- The remaining balance of the loans is calculcated monthly up to 6 months in the future
- The monthly installment is subtracted from the remaining balance to calculcate the remaining balance for each month
- The remaining balance is summed for each month to calculate the percentage recovery
- This projection is visualised with a scatter plot

In [ ]:
# visualise what percentage of the total amount would be recovered up to 6 months' in the future.
loan_payments_recovery_projection_df = loan_payments_df.copy()
loan_payments_recovery_projection_df['remaining_balance'] = loan_payments_recovery_projection_df['loan_amount'] -  loan_payments_recovery_projection_df['total_payment'] # calculate remaining balance of each loan
loan_payments_recovery_projection_df['month1'] = loan_payments_recovery_projection_df['remaining_balance'] - loan_payments_recovery_projection_df['instalment'] # minus monthly payment to get remaining balance for that month
loan_payments_recovery_projection_df['month2'] = loan_payments_recovery_projection_df['remaining_balance'] - (loan_payments_recovery_projection_df['instalment']*2) 
loan_payments_recovery_projection_df['month3'] = loan_payments_recovery_projection_df['remaining_balance'] - (loan_payments_recovery_projection_df['instalment']*3)
loan_payments_recovery_projection_df['month4'] = loan_payments_recovery_projection_df['remaining_balance'] - (loan_payments_recovery_projection_df['instalment']*4)
loan_payments_recovery_projection_df['month5'] = loan_payments_recovery_projection_df['remaining_balance'] - (loan_payments_recovery_projection_df['instalment']*5)
loan_payments_recovery_projection_df['month6'] = loan_payments_recovery_projection_df['remaining_balance'] - (loan_payments_recovery_projection_df['instalment']*6)

month1_pc = (((loan_payments_recovery_projection_df.month1 <= 0.00).sum())/(len(loan_payments_df)))*100 # calculate percentage of loans recovered after each month
month2_pc = (((loan_payments_recovery_projection_df.month2 <= 0.00).sum())/(len(loan_payments_df)))*100
month3_pc = (((loan_payments_recovery_projection_df.month3 <= 0.00).sum())/(len(loan_payments_df)))*100
month4_pc = (((loan_payments_recovery_projection_df.month4 <= 0.00).sum())/(len(loan_payments_df)))*100
month5_pc = (((loan_payments_recovery_projection_df.month5 <= 0.00).sum())/(len(loan_payments_df)))*100
month6_pc = (((loan_payments_recovery_projection_df.month6 <= 0.00).sum())/(len(loan_payments_df)))*100

month_no = [1,2,3,4,5,6]
loans_recovered_pc = [month1_pc, month2_pc, month3_pc, month4_pc, month5_pc, month6_pc]
recovery_df = pd.DataFrame({'Month': month_no,'Recovery_percentage': loans_recovered_pc}) # create df to plot data 
fig2 = plt.figure(2)
sns.scatterplot(data=recovery_df, x='Month', y='Recovery_percentage') # scatter plot of data to visualise projection

### 2.2.1 Calculate the percentage of charged off loans historically 
- Charged off loans represent a loss to the company
- The percentage of loans marked as charged off is calculated using the loan_status categorisation 


In [ ]:
# Calculate the percentage of charged off loans historically 
charged_off_loans = (loan_payments_df.loan_status == 'Charged Off').sum()  
charged_off_loans_pc = (charged_off_loans/(len(loan_payments_df)))*100
print(f"Percentage of charged off loans historically = {charged_off_loans_pc.round(2)} %")

### 2.2.2 Calculate the total amount paid towards these loans before being charged off
- Charged off loans are ifentified and the amount paid towards the loans is calculated
- The amount paid is summed and divided by the sum of the total loan amounts to obtain a percentage

In [ ]:
# and the total amount that was paid towards these loans before being charged off.
charged_off_loans_df = loan_payments_df.loc[loan_payments_df['loan_status']=='Charged Off']
total_loan_amount = charged_off_loans_df['loan_amount'].sum()
total_amount_paid = charged_off_loans_df['total_payment'].sum()
total_amount_paid_pc = (total_amount_paid/total_loan_amount)*100
print(f"total amount that was paid towards these loans before being charged off = £{total_amount_paid.round(2)}")

### 2.2.3 Calulcate the loss in revenue generated if these loans finished their term
- The time between the loan issue date and last payment date is calculated and converted to months 
- The loan payment term remaining is calculcated by subtracting the monthly payments paid so far from the term of the loan
- The amount remaining to be paid for the loans is calculated from the remaining months on the loan term multiplied by the monthly payment
- This gives the total potential revenue lost from loans that have been charged off 

In [ ]:
# Calculate the loss in revenue these loans would have generated for the company if they had finished their term. 
time_passed = ((charged_off_loans_df['last_payment_date'] - charged_off_loans_df['issue_date'])) # calculate how much time has passed of the loan term
time_passed_days = time_passed.dt.days # convert to days 
charged_off_loans_df_calc = charged_off_loans_df.copy()
charged_off_loans_df_calc['time_passed_months'] = (time_passed_days/30.5).round() # convert to int value and divide by avg month length to get no of months
charged_off_loans_df_calc['time_remaining'] = charged_off_loans_df_calc['term'] - charged_off_loans_df_calc['time_passed_months'] # calculate number of months left of the term
charged_off_loans_df_calc['lost_revenue'] = charged_off_loans_df_calc['time_remaining']*charged_off_loans_df_calc['instalment'] # amount that would be paid over the remaining term
total_revenue_loss = charged_off_loans_df_calc['lost_revenue'].sum()
print(f"total loss in revenue these loans would have generated for the company  = £{total_revenue_loss.round(2)}")

### 2.3.1 Calculate the % of users behind with loan payments
- Users behind with loan payments are categorised as Late (16-30 days) or Late (31-120 days)
- Users with loans in these categories are summed 
- This value is divided by the total number of loans to obtain a percentage value

In [ ]:
# calculate the percentage of users behind with loan payments 
late_loans_1 = (loan_payments_df.loan_status == 'Late (16-30 days)').sum()   # count no of loans marked as late
late_loans_2 = (loan_payments_df.loan_status == 'Late (31-120 days)').sum()   # count no of loans marked as late
late_loans_pc = ((late_loans_1 + late_loans_2)/(len(loan_payments_df)))*100 # calculate total % of late loans 
print(f"total percentage of users behind with loan payments  = {late_loans_pc.round(2)} %")


### 2.3.2 Calculate the loss to the company if loan status of these users was changed to charged off 
- The reminaing amount owed is summed for users who are behind with loans to calcualte the potential loss in revenue

In [ ]:
# calculate how much loss the company would incur their status was changed to Charged Off
late_loans_df = loan_payments_df.apply(lambda row: row[loan_payments_df['loan_status'].isin(['Late (16-30 days)','Late (31-120 days)'])]) # filter df to obtain only rows with late status 
loss_incurred = late_loans_df['out_prncp'].sum() # sum the outstanding amount for each late loan to calculate loss 
print(f"total loss the company would incur if their status was changed to Charged Off  = £{loss_incurred.round(2)}")

### 2.3.3 Calculated the projected loss if these customers were to finish their full loan term
- The time between the loan issue date and last payment date is calculated and converted to months 
- The loan payment term remaining is calculcated by subtracting the monthly payments paid so far from the term of the loan
- The remaining amount that would be paid towards the loans is calculated from the remaining months on the loan term multiplied by the monthly payment
- The projected loss is calculated as the outstanding amount on the loans after the full payment term is completed

In [ ]:
# What is the projected loss of these loans if the customer were to finish the full loans term?
time_passed_lateloans = ((late_loans_df['last_payment_date'] - late_loans_df['issue_date'])) # calculate how much time has passed of the loan term
time_passed_days_lateloans = time_passed_lateloans.dt.days # convert to days 
late_loans_df['time_passed_months'] = (time_passed_days_lateloans/30.5).round() # convert to int value and divide by avg month length to get no of months
late_loans_df['time_remaining'] = late_loans_df['term'] - late_loans_df['time_passed_months'] # calculate number of months left of the term
late_loans_df['outstanding_payments'] = late_loans_df['time_remaining']*late_loans_df['instalment'] # amount that would be paid over the remaining term
projected_loss = late_loans_df['outstanding_payments'].sum() 
print(f"total projected loss of these loans if the customers were to finish the full loans term  = £{projected_loss.round(2)}")

### 2.3.4 If customers late on payments converted to Charged Off, what percentage of total expected revenue do these customers and the customers who have already defaulted on their loan represent?
- A subset of data is created including all users who are late on payments and those who have had their loans charged off
- Monthly revenue from this subset of users is summed and divided by total monthly revenue to calculcate the percentage

In [ ]:
# If customers late on payments converted to Charged Off, what percentage of total expected revenue do these customers and the customers who have already defaulted on their loan represent?
lost_revenue_df = loan_payments_df.apply(lambda row: row[loan_payments_df['loan_status'].isin(['Late (16-30 days)','Late (31-120 days)','Charged Off'])]) # filters customers who have already defaulted on loans as well as those with late payment status 
monthly_revenue_late_customers = lost_revenue_df['instalment'].sum()
total_monthly_revenue = loan_payments_df['instalment'].sum()
total_revenue_pc = (monthly_revenue_late_customers/total_monthly_revenue)*100
print(f"total percentage of total expected revenue from late and stopped payments  = {total_revenue_pc.round(2)} %")

### 2.4 Visualise the data to examine possible indicators of loss
- The data is visualised to see if there are possible indicators that suggest that a customer will be unable to pay their loan, leading to losses for the company
- Subsets of the data are created, a subset containing those late on loan payments, and another subset containing those with charged off loans who are no longer paying. This enables comparison between categories of users. 
- Instances of the Plotter class and DataFrameInfo class are initialised for the main dataframe and each subset of the data

In [ ]:
# in order to visualise the possible indicators that a customer will not be able to pay the loan.
loan_payments_df_stopped_paying = loan_payments_df.loc[loan_payments_df['loan_status']=='Charged Off'] # subset containing only customers who have stopped paying
loan_payments_df_late_payments = loan_payments_df.apply(lambda row: row[loan_payments_df['loan_status'].isin(['Late (16-30 days)','Late (31-120 days)'])]) # subset containings customers with late payments

# initialise instances of plotter class for each df
original_df_plotter = Plotter(loan_payments_df)
stopped_paying_df_plotter = Plotter(loan_payments_df_stopped_paying)
late_payments_df_plotter = Plotter(loan_payments_df_late_payments)

# intiailise instances of dataframeinfo class for each df
original_df_info = DataFrameInfo(loan_payments_df)
stopped_paying_df_info = DataFrameInfo(loan_payments_df_stopped_paying)
late_payments_df_info = DataFrameInfo(loan_payments_df_late_payments)

### 2.4.1 Grade of the loan
- Countplots are generated to visualise if the loan grade is an indicator of loss
- No signficant change is seen in the distribution for each data set, indicating this is not an indicator of loss

In [ ]:
#grade of the loan as an indicator - NOT STRONG INDICATOR
fig3 = plt.figure(3)
original_df_plotter.plot_countplot('grade') # visualise distribution for all data
fig4 = plt.figure(4)
stopped_paying_df_plotter.plot_countplot('grade')  # visualise distribution for customers who are not paying
fig5 = plt.figure(5)
late_payments_df_plotter.plot_countplot('grade') # visualise distribution for customers who are late paying


### 2.4.2 Purpose of the loan
- Countplots are generated to visualise if the loan purpose is an indicator of loss
- No signficant change is seen in the distribution for each data set, indicating this is not an indicator of loss

In [ ]:
# purpose of loan as indicator - NOT STRONG INDICATOR
fig6 = plt.figure(6)
original_df_plotter.plot_countplot('purpose') # visualise distribution for all data
fig7 = plt.figure(7)
stopped_paying_df_plotter.plot_countplot('purpose')  # visualise distribution for customers who are not paying
fig8 = plt.figure(8)
late_payments_df_plotter.plot_countplot('purpose') # visualise distribution for customers who are late paying


### 2.4.3 Home ownership
- Countplots are generated to visualise if home ownership status is an indicator of loss
- No signficant change is seen in the distribution for each data set, indicating this is not an indicator of loss

In [ ]:
# home owenership as indicator - NOT STRONG INDICATOR
fig9 = plt.figure(9)
original_df_plotter.plot_countplot('home_ownership') # visualise distribution for all data
fig10 = plt.figure(10)
stopped_paying_df_plotter.plot_countplot('home_ownership')  # visualise distribution for customers who are not paying
fig11 = plt.figure(11)
late_payments_df_plotter.plot_countplot('home_ownership') # visualise distribution for customers who are late paying


### 2.4.4 Remaining amount of the loan (out_prncp)
- KDE plots are generated to visualise if the outstanding loan amount is an indicator of loss
- The mean remaining principal is calculated for each set of users
- Those with late payments have a higher average remaining principal, suggesting that out_prncp is a possible loss indicator

In [ ]:
#out_prncp as indicator - POSSIBLE INDICATOR
fig12 = plt.figure(12)
original_df_plotter.plot_KDE('out_prncp') # visualise distribution for all data
fig13 = plt.figure(13)
stopped_paying_df_plotter.plot_KDE('out_prncp')  # visualise distribution for customers who are not paying
fig14 = plt.figure(14)
late_payments_df_plotter.plot_KDE('out_prncp') # visualise distribution for customers who are late paying

original_mean_out_prncp = original_df_info.get_mean('out_prncp') # caluclate mean for all data
stopped_paying_mean_out_prncp = stopped_paying_df_info.get_mean('out_prncp') # calcualte mean for customers who are not paying
late_payment_mean_out_prncp = late_payments_df_info.get_mean('out_prncp') # calcualte mean for customers who are late paying
print(original_mean_out_prncp, stopped_paying_mean_out_prncp, late_payment_mean_out_prncp) 
# higher average out_prncp for those with late payments, suggests this is an indicator for late payments 

### 2.4.4 Late fees recieved (total_rec_late_fee)
- KDE plots are generated to visualise if the total late fees are an indicator of loss
- The mean late fee is calculated for each set of users
- Those with late payments or charged off loans have a higher average total late fees, suggests that total_rec_late_fee is a possible loss indicator 

In [ ]:
# total_rec_late_fee as indicator - POSSIBLE INDICATOR
fig15 = plt.figure(15)
original_df_plotter.plot_KDE('total_rec_late_fee') # visualise distribution for all data
fig16 = plt.figure(16)
stopped_paying_df_plotter.plot_KDE('total_rec_late_fee')  # visualise distribution for customers who are not paying
fig17 = plt.figure(17)
late_payments_df_plotter.plot_KDE('total_rec_late_fee') # visualise distribution for customers who are late paying
original_mean_total_rec_late_fee = original_df_info.get_mean('total_rec_late_fee') # caluclate mean for all data
stopped_paying_mean_total_rec_late_fee = stopped_paying_df_info.get_mean('total_rec_late_fee') # calcualte mean for customers who are not paying
late_payment_mean_total_rec_late_fee = late_payments_df_info.get_mean('total_rec_late_fee') # calcualte mean for customers who are late paying
print(original_mean_total_rec_late_fee, stopped_paying_mean_total_rec_late_fee, late_payment_mean_total_rec_late_fee) 
# higher average total late fees for those with late payments or charged off payments, suggests this is an indicator 

### 2.4.5 Interest paid (total_rec_int)
- KDE plots are generated to visualise if the total interest paid is an indicator of loss
- The mean interest paid is calculated for each set of users
- Those with late payments have a higher average total interest paid, suggests that total_rec_int is a possible loss indicator 

In [ ]:
# total_rec_int as indicator - POSSIBLE INDICATOR
fig18 = plt.figure(18) 
original_df_plotter.plot_KDE('total_rec_int') # visualise distribution for all data
fig18 = plt.figure(18)
stopped_paying_df_plotter.plot_KDE('total_rec_int') # visualise distribution for customers who are not paying
fig18 = plt.figure(18)
late_payments_df_plotter.plot_KDE('total_rec_int') # visualise distribution for customers who are late paying
original_mean_total_rec_int = original_df_info.get_mean('total_rec_int') # caluclate mean for all data
stopped_paying_mean_total_rec_int = stopped_paying_df_info.get_mean('total_rec_int') # calcualte mean for customers who are not paying
late_payment_mean_total_rec_int= late_payments_df_info.get_mean('total_rec_int') # calcualte mean for customers who are late paying
print(original_mean_total_rec_int, stopped_paying_mean_total_rec_int, late_payment_mean_total_rec_int) 
# higher averages for late payments suggests total_rec_int is an indicator 

### 2.4.6 Last payment amount
- KDE plots are generated to visualise if the last payment amount is an indicator of loss
- The mean last payment amount is calculated for each set of users
- Those with late payments or charged off loans have a lower average last payment amount, which suggests that last payment amount is a possible loss indicator 

In [ ]:
#last_payment_amount as indicator - POSSIBLE INDICATOR
fig19 = plt.figure(19)
original_df_plotter.plot_KDE('last_payment_amount') # visualise distribution for all data
fig20 = plt.figure(20)
stopped_paying_df_plotter.plot_KDE('last_payment_amount') # visualise distribution for customers who are not paying
fig21 = plt.figure(21)
late_payments_df_plotter.plot_KDE('last_payment_amount') # visualise distribution for customers who are late paying
original_mean_last_payment_amount = original_df_info.get_mean('last_payment_amount') # caluclate mean for all data
stopped_paying_mean_last_payment_amount = stopped_paying_df_info.get_mean('last_payment_amount') # calcualte mean for customers who are not paying
late_payment_mean_last_payment_amount= late_payments_df_info.get_mean('last_payment_amount') # calcualte mean for customers who are late paying
print(original_mean_last_payment_amount, stopped_paying_mean_last_payment_amount, late_payment_mean_last_payment_amount)
# Last payments amount is lower for late payments and charged off payments, suggests lower last payment amount is an indicator of not paying back/paying late


### 2.4.7 Possible loss indicators summary
- The following columns can potentially be used to indicate possible losses
- Remaining amount of the loan (out_prncp): those with late payments have a higher average remaining principal
- Late fees recieved (total_rec_late_fee): those with late payments or charged off loans have a higher average total late fees
- Interest paid (total_rec_int): those with late payments have a higher average total interest paid
- Last payment amount (lsat_payment_amount): those with late payments or charged off loans have a lower average last payment amount,